# Case Study: Home Depot, Lowes, Tool Time

### Initial data load

In [ ]:
import pandas as pd
import numpy as np

# County codes for AK, AL, AR, AZ, CA, CO, and CT properly formatted as strings with "0" prefixes to be read as 5 digits
# See full explanation of method here: 
# https://stackoverflow.com/questions/20025882/add-a-string-prefix-to-each-value-in-a-string-column-using-pandas
hdl_data = pd.read_csv('JC_clean_data/JC_clean_Home_Depot_Lowes_Data.csv', dtype={"county": str})

In [ ]:
pd.set_option('display.max_columns', 27)

In [ ]:
hdl_data.head()

### Question 1: Perform Exploratory Data Analysis on the stores

**a. What are the total store counts of Home Depot and Lowes?**

In [ ]:
# Home Depot store count
hdl_data.HDcount.sum()

In [ ]:
# Lowes store count
hdl_data.Lcount.sum()

**b. Create one dummy variable for Home Depot and one dummy variable for Lowes that identifies if the store is located in a county**

In [ ]:
hdl_data['HDexists'] = [1 if x > 0 else 0 for x in hdl_data.HDcount]
hdl_data['Lexists'] = [1 if x > 0 else 0 for x in hdl_data.Lcount]

**c. Which store is present in more counties?**

In [ ]:
# Sum of counties with Home Depot present
hdl_data.HDexists.sum()

In [ ]:
# Sum of counties with Lowes present
hdl_data.Lexists.sum()

Lowes stores are present in more counties than Home Depot stores

### Question 2: Use a United States map with FIPS locations to plot the store locations of both Lowes and Home Depot

In [ ]:
# Install these packages and dependencies:
# conda install -c plotly plotly-geo
# conda install -c conda-forge pyshp
# conda install -c conda-forge geopandas

# This package is going to be deprecated and therefore should not be used
# from plotly.figure_factory._county_choropleth import create_choropleth

from plotly.offline import init_notebook_mode, plot, iplot
import plotly.express as px
import geopandas

init_notebook_mode(connected=True)

In [ ]:
# plotly and geopandas necessary for producing these choropleths
# For plotly express to print maps, jsons must be used instead of FIPS values for producting county shapes
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

# These values do not need to be cast in lists anymore  
# hd_values = hdl_data.HDcount.tolist()
# l_values = hdl_data.Lcount.tolist()
# fips = hdl_data.county.tolist()

hd_fig = px.choropleth_mapbox(hdl_data, geojson=counties, locations='county', color='HDcount',
                            color_continuous_scale="Viridis",
                            range_color=(0, 12),
                            mapbox_style="carto-positron",
                            zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                            opacity=0.5,
                            labels={'HDcount':'Number of Stores'})

l_fig = px.choropleth_mapbox(hdl_data, geojson=counties, locations='county', color='Lcount',
                            color_continuous_scale="Viridis",
                            range_color=(0, 12),
                            mapbox_style="carto-positron",
                            zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                            opacity=0.5,
                            labels={'Lcount':'Number of Stores'})


# Obsolete figure factory map production methods never to be used again 
# hd_fig = create_choropleth(
#             fips=fips, values=hd_values, show_state_data=True,
#             county_outline={'color': 'rgb(255,255,255)', 'width': 0.5},
#             title='United States Home Depot Store Locations by County',
#             legend_title='Number of Stores')

# l_fig = create_choropleth(
#             fips=fips, values=l_values, show_state_data=True,
#             county_outline={'color': 'rgb(255,255,255)', 'width': 0.5},
#             title='United States Lowes Store Locations by County',
#             legend_title='Number of Stores')

**a. What observations can you make from the map?**

In [ ]:
iplot(hd_fig)

# If iplot doesn't show a figure, uncomment and run the code below
# plot(hd_fig)

In [ ]:
iplot(l_fig)

# If iplot doesn't show a figure, uncomment and run the code below
# plot(l_fig)

It looks like Mid to Southern California, Southwestern Arizona, and Florida are promissing places to open home improvement stores. Texas and Washington also have some counties that should be considered. There seems to be at least one county in most states that have a high number of Home Depot or Lowes stores.

### Question 3: Create a linear regression model to identify the correlations among the variables.

#### Clean the data

**Impute missing values**

In [ ]:
hdl_data[hdl_data.isna().any(axis=1)]

Rows 1654, 2922, and 2950 are missing most data, so we should drop them.

In [ ]:
hdl_data.drop([1654,2922,2950], inplace=True)

It looks like these missing values exist because there was no population data in 2000. We can replace these missing values with the mean of the feature.

In [ ]:
hdl_data[hdl_data.isna().any(axis=1)]

In [ ]:
hdl_data.pct_U18_2000.fillna(hdl_data.pct_U18_2000.mean(), inplace=True)
hdl_data.pctwhite_2000.fillna(hdl_data.pctwhite_2000.mean(), inplace=True)
hdl_data.pctblack_2000.fillna(hdl_data.pctblack_2000.mean(), inplace=True)

In [ ]:
# Split data into HD and L, keep only demographic data columns and target, our target for linear regression
# is the numeric "count" feature
hd_data = hdl_data.copy()
l_data = hdl_data.copy()

hd_data.drop(['areaname', 'county', 'state', 'r1', 'r2', 'Lcount', 'HDexists', 'Lexists'], axis=1, inplace=True)
l_data.drop(['areaname', 'county', 'state', 'r1', 'r2', 'HDcount', 'HDexists', 'Lexists'], axis=1, inplace=True)

hd_features = hd_data.drop(['HDcount'], axis=1)
hd_target = hd_data.HDcount

l_features = l_data.drop(['Lcount'], axis=1)
l_target = l_data.Lcount

In [ ]:
from sklearn.linear_model import LinearRegression
# Scale the data using Pipeline with a StandardScaler in a preprocessing stage
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Create the pipelines for both models set with LinearRegression() for both
hd_lm = make_pipeline(StandardScaler(with_mean=False), LinearRegression())
l_lm = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

# Fit the models correspondingly
hd_lm.fit(hd_features, hd_target)
l_lm.fit(l_features, l_target)

# Obsolete method soon to be deprecated
# hd_lm = LinearRegression(normalize=True).fit(hd_features, hd_target)
# l_lm = LinearRegression(normalize=True).fit(l_features, l_target)

In [ ]:
from sklearn.model_selection import cross_val_score

print(cross_val_score(hd_lm, hd_features, hd_target, cv=5))
print(cross_val_score(l_lm, l_features, l_target, cv=5))

**a. What customer demographic variables are most import to Lowes?**

In [ ]:
# Access coefficients in make_pipeline model by calling steps[1][1] as explained here:
# https://stackoverflow.com/questions/34373606/scikit-learn-coefficients-polynomialfeatures
l_coeficients = zip(list(l_features), l_lm.steps[1][1].coef_)
sorted(list(l_coeficients), key=lambda x: x[1], reverse=True)

Lowes is very interested in building stores in locations where there is a high percentage of people under the age of 18. Maybe this is an indicator that new families live in these areas and will be investing a lot of time and money into their homes over a longer period of time.

**b. What customer demographic variables are most import to Home Depot?**

In [ ]:
# Access coefficients in make_pipeline model by calling steps[1][1] as explained here:
# https://stackoverflow.com/questions/34373606/scikit-learn-coefficients-polynomialfeatures
hd_coeficients = zip(list(hd_features), hd_lm.steps[1][1].coef_)
sorted(list(hd_coeficients), key=lambda x: x[1], reverse=True)

Home Depot is interested in the percentage of different races of people living in the area. This chain also values percentage of people in college.

**c. How are the chains similar in their decision making?**

Both chains seem to use percentage of people of different race in their decision making. They also seem to be interested in areas where there is a higher percentage of young people.

**d. How are they different?**

Home Depot values home owners percentage in 2000 higher than Lowes. In addition, Lowes seems to value 2000 density more than Home Depot. It's interesting that both chains values the 2000 demographic statistics over the 2010 data.

### Question 4:	What are the top 5 towns / cities that can be predicted as potential candidates for new locations for both Lowes and Home Depot? 

**Prepare data**

In [ ]:
# Split data into HD and L, keep only demographic data columns and target, our target for logistic regression
# is the boolean "exists" feature
hd_data = hdl_data.copy()
l_data = hdl_data.copy()

hd_data.drop(['areaname', 'county', 'state', 'r1', 'r2', 'Lcount', 'HDcount', 'Lexists'], axis=1, inplace=True)
l_data.drop(['areaname', 'county', 'state', 'r1', 'r2', 'Lcount', 'HDcount', 'HDexists'], axis=1, inplace=True)

hd_X = hd_data.drop(['HDexists'], axis=1)
hd_y = hd_data.HDexists

l_X = l_data.drop(['Lexists'], axis=1)
l_y = l_data.Lexists

from sklearn.model_selection import train_test_split

# Split the data into train and test portions to test accuracy
hd_X_train, hd_X_test, hd_y_train, hd_y_test = train_test_split(hd_X, hd_y, random_state=42)
l_X_train, l_X_test, l_y_train, l_y_test = train_test_split(l_X, l_y, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

hd_logit = LogisticRegressionCV(Cs=10, cv=5, class_weight='balanced', max_iter=1000, random_state=42).fit(hd_X_train, hd_y_train)
l_logit = LogisticRegressionCV(Cs=10, cv=5, class_weight='balanced', max_iter=1000, random_state=42).fit(l_X_train, l_y_train)

In [ ]:
# Home Depot store predicition accuracy
hd_logit.score(hd_X_test, hd_y_test)

In [ ]:
# Lowes store predicition accuracy
l_logit.score(l_X_test, l_y_test)

Accuracy for predicting the test portions of the data looks pretty solid so we can train on the whole data to predict the next best store locations.

In [ ]:
hd_full_logit = LogisticRegressionCV(Cs=5, cv=5, class_weight='balanced', max_iter=1000, random_state=42).fit(hd_X, hd_y)
l_full_logit = LogisticRegressionCV(Cs=5, cv=5, class_weight='balanced', max_iter=1000, random_state=42).fit(l_X, l_y)

In [ ]:
# Predict the target for all locations and extract the probability that a store should be built
hd_store_prob = [x[1] for x in hd_full_logit.predict_proba(hd_X)]
l_store_prob = [x[1] for x in l_full_logit.predict_proba(l_X)]

In [ ]:
# Add the probability features to the original dataset so we can find the best next locations to build stores
hdl_data['hd_store_prob'] = hd_store_prob
hdl_data['l_store_prob'] = l_store_prob

In [ ]:
# For Home Depot, show only locations where there are no Home Depot stores,
# sort by hd_store_prob descending and show top 5 areas predicted to be the best store locations
hdl_data[hdl_data.HDexists == 0].sort_values(by='hd_store_prob', ascending=False).head(10)

The logistic regression model predicts that San Francisco, LA; Union, NC; Arlington, VA; Elkhart, IN; and Sangamon, IL are the top 5 locations where Home Depot is **not** already located to build new stores. If Home Depot is concerned about competition from Lowes stores in the area, we could also inspect locations where neither Home Depot nor Lowes stores exist. But these top areas have few stores, with the exception of Union, NC.

In [ ]:
# For Lowes, show only locations where there are no Lowes stores,
# sort by l_store_prob descending and show top 5 areas predicted to be the best store locations
hdl_data[hdl_data.Lexists == 0].sort_values(by='l_store_prob', ascending=False).head(10)

The logistic regression model predicts that Westchester, NY; Dane, WI; Denver, CO; Essex, NJ; and Santa Barbara, CA are the top 5 locations where Lowes is **not** already located to build new stores. These locations seem, especially Westchester, NY seem to be saturated with Home Depot stores, so it may be wiser to choose locations with lower probabilities but less competition from the other chain.

## Question 5. Where should “Tool Time” build its next 5 stores based on the Census Data on your customers? 

In [ ]:
# Create new column that is the sum of the two probability columns for sorting purposes
hdl_data['prob_sum'] = hdl_data.hd_store_prob + hdl_data.l_store_prob

In [ ]:
# For Tool Time, show only locations where there are 1 or no stores for HD and Lowes,
# sort by both l_store_prob and hd_store_prob descending and show top 5 areas predicted to be the best store locations
hdl_data[(hdl_data.Lcount <= 1) & (hdl_data.HDcount <= 1)].sort_values(by='prob_sum', ascending=False).head(10)

The logistic regression model predicts that Pinal, AZ; Ramsey, MN; Weld, CO; Webb, TX; and Ingham, MI are the top 5 locations to build new Tool Time stores.

**a. Explain your rational for your decision**

We want to find locations where both Home Depot and Lowes would consider building new stores, but we don't want to build in an area that is already saturated with other chains' stores. For this reason, I filtered the data by locations that had 1 or no stores for both chains. Then I sorted by probability that the location is a good place to build a store. This gives us locations that are promising for building stores, but have low stauration in terms of competition.

## Question 6. realtor.com market hotness index report 

**a. Using the realtor.com market hotness index report from August of 2018 create an additional variable to segment the country into the following regions**

In [ ]:
# County codes for AK, AL, AR, AZ, CA, CO, and CT properly formatted as strings with "0" prefixes to be read as 5 digits
# See full explanation of method here: 
# https://stackoverflow.com/questions/20025882/add-a-string-prefix-to-each-value-in-a-string-column-using-pandas
# For dropping unwanted "Unnamed 0" index column, see here:
# https://stackoverflow.com/questions/36519086/how-to-get-rid-of-unnamed-0-column-in-a-pandas-dataframe-read-in-from-csv-fil

# Load the data
state_region = pd.read_csv('JC_clean_data/state_region.csv', dtype={"State Code": str})
realtor_data = pd.read_csv('JC_clean_data/JC_clean_RDC_MarketHotness_Monthly.csv', dtype={"CountyFIPS": str, "State Code": str})

# Keep only relevant columns from state_region
state_region.drop(['State', 'Division'], axis=1, inplace=True)

import re

# Drop the United States row
realtor_data = realtor_data[realtor_data.CountyName != 'United States']
# Use regex to extract the state abbreviation from "ZipName" in realtor_data and make a new column
realtor_data['State Code'] = [re.search('\w+, ([A-Z]{2})', x).group(1) for x in realtor_data.ZipName]

In [ ]:
# Properly merge the two dataframes by filtering out duplicates, since the copy=False parameter was not working
# Explained here: https://stackoverflow.com/questions/19125091/pandas-merge-how-to-avoid-duplicating-columns

# Join the state_region and realtor_data dataframes on 'State Code'
realtor_data = realtor_data.merge(state_region, how='left', on='State Code', 
                                  suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

In [ ]:
# Note the new last column
realtor_data

In [ ]:
# Drop duplicate FIPS codes, keep first
realtor_data.drop_duplicates(subset='CountyFIPS', inplace=True)

In [ ]:
# Reset the index and show new data
realtor_data.reset_index(drop=True, inplace=True)
realtor_data

**b. Exploratory Data Analysis for realator.com data**

**i. Which Region of the country has the best “Demand Score”**

In [ ]:
# Group by "Region", find mean of "Demand Score"
realtor_data.groupby('Region')['Demand Score'].agg(np.mean).sort_values(ascending=False)

The Northeast has the best mean "Demand Score."

**ii. Which State in the country has the best “Demand Score”**

In [ ]:
# Group by "State Code", find mean of "Demand Score"
realtor_data.groupby('State Code')['Demand Score'].agg(np.mean).sort_values(ascending=False).head()

Massachusetts has the best mean "Demand Score."

**iii. Which metro area (pop_2010 > 1million) has the best “Demand Score”**

In [ ]:
# Using the FIPS codes, join the POP_2010 column from hdl_data to realtor_data
pop_data = hdl_data[['county', 'pop_2010']]
realtor_withpop = pd.merge(realtor_data, pop_data, how='inner', left_on='CountyFIPS', right_on='county')

# Filter by pop_2010 > 1mil, then find highest "Demand Score"
realtor_withpop[realtor_withpop.pop_2010 > 1e6][['CountyName', 'State Code','Demand Score']]

Middlesex, MA has the best "Demand Score" of metro areas.

**c.	Compare and contrast these findings with your predicted new store findings.**

**i. Describe your findings as they relate to the customer attributes and potential business opportunity that Lowes, Home Depot and/or Tool Time may have if they are or are not located in the areas that have high demands for real estate opportunities**

It appears that the Northeast, Texas, Ohio, California, and Florida are all good areas for stores to be loacted, according to "Demand Score." This somewhat agrees with the model predictions; there were a lot of predicted locations in California and the Northeast.

**d. Add the following as features to the original HDLo data set and predict again where Tool Time should build its next 5 stores.**
1.	Median.Listing.Price
2.	Demand.Score
3.	Hotness.Score
4.	Nieleson.HH.Rank

In [ ]:
# Subset the realtor_data to only contain the columns we want to merge with the hdl_data
for_merge = realtor_data[['CountyFIPS', 'Median Listing Price', 'Demand Score', 'Hotness Score', 'Nielsen HH Rank']]

# Rename columns
for_merge.columns = ['county', 'Median.Listing.Price', 'Demand.Score', 'Hotness.Score', 'Nieleson.HH.Rank']

# Merge this data with hdl_data on county
added_data = pd.merge(hdl_data, for_merge, how='inner', on='county', copy=False)
# Drop previous store probability columns
added_data.drop(columns=['hd_store_prob', 'l_store_prob', 'prob_sum'], inplace=True)

In [ ]:
hdl_data

In [ ]:
# Copy data into training and testing sets, drop earlier added probability columns along with other irrelevant columns
hd_data = added_data.copy()
l_data = added_data.copy()

hd_data.drop(['areaname', 'county', 'state', 'r1', 'r2', 'Lcount', 'HDcount', 'Lexists'], axis=1, inplace=True)
l_data.drop(['areaname', 'county', 'state', 'r1', 'r2', 'Lcount', 'HDcount', 'HDexists'], axis=1, inplace=True)

hd_X = hd_data.drop(['HDexists'], axis=1)
hd_y = hd_data.HDexists

l_X = l_data.drop(['Lexists'], axis=1)
l_y = l_data.Lexists

# Split the data into train and test portions to test accuracy
hd_X_train, hd_X_test, hd_y_train, hd_y_test = train_test_split(hd_X, hd_y, random_state=42)
l_X_train, l_X_test, l_y_train, l_y_test = train_test_split(l_X, l_y, random_state=42)

In [ ]:
# Train the models
hd_logit = LogisticRegressionCV(Cs=10, cv=5, class_weight='balanced', max_iter=10000, random_state=42).fit(hd_X_train, hd_y_train)
l_logit = LogisticRegressionCV(Cs=10, cv=5, class_weight='balanced', max_iter=10000, random_state=42).fit(l_X_train, l_y_train)

In [ ]:
# Home Depot store predicition accuracy
hd_logit.score(hd_X_test, hd_y_test)

In [ ]:
# Lowes store predicition accuracy
l_logit.score(l_X_test, l_y_test)

In [ ]:
# Train on the full data
hd_full_logit = LogisticRegressionCV(Cs=10, cv=5, class_weight='balanced', max_iter=15000, random_state=42).fit(hd_X, hd_y)
l_full_logit = LogisticRegressionCV(Cs=10, cv=5, class_weight='balanced', max_iter=15000, random_state=42).fit(l_X, l_y)

In [ ]:
# Predict the target for all locations and extract the probability that a store should be built
hd_store_prob = [x[1] for x in hd_full_logit.predict_proba(hd_X)]
l_store_prob = [x[1] for x in l_full_logit.predict_proba(l_X)]

In [ ]:
# Add the probability features to the original dataset so we can find the best next locations to build stores
added_data['hd_store_prob'] = hd_store_prob
added_data['l_store_prob'] = l_store_prob

# Create new column that is the sum of the two probability columns for sorting purposes
added_data['prob_sum'] = added_data.hd_store_prob + added_data.l_store_prob

In [ ]:
# For Tool Time, show only locations where there are 1 or no stores for HD and Lowes,
# sort by prob_sum descending and show top 5 areas predicted to be the best store locations
added_data[(added_data.Lcount <= 1) & (added_data.HDcount <= 1)].sort_values(by='prob_sum', ascending=False).head(10)

**e. What are the top 5 new area names for which Tool Time should build their stores?**

The first four stores remain the same, however Ottawa, MI overtook Ingham, MI in this version of the model.

**f. Do these features increase the prediction accuracy for the new area predictions?**

Based on the new scores, the new features seem to have decreased the accuracy of the models.

**g. Does overlaying the realtor data set add value to the business strategy of Tool Time?**

In this case, I would conclude that adding the new features **does not** add value to the business strategy that I would recommend.

**h. Is there an alternative strategy that Tool Time should explore other than Census Data and Realtor data?**

Tool Time might want to consider areas that have high demand for commerical real estate, as new businesses will have contractors in need of supplies. Tool Time might also want to look into trends for up-and-coming neighborhoods where real estate prices may increase in the future, and get ahead of competitors by opening stores newly desired neighborhoods.